In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_8268\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"C:\Users\Gebruiker\Desktop\IronHack\Labs\week 7\lab-natural-language-processing\data\kg_train.csv", encoding='latin-1')

#data = pd.read_csv("data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [4]:
data.head(20)

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0
5,sure -- bottom line - you need a special secur...,0
6,"Dear Sir,I am Engr. Ugo Nzego with the Enginee...",1
7,Abedin Huma <AbedinH@state.gov>Saturday Novemb...,0
8,There is an Oct 16th George Marshall event at ...,0
9,<P>1 25% for you as the account owner <BR>2 65...,1


### Let's divide the training and test set into two partitions

In [5]:
data_test = pd.read_csv(r"C:\Users\Gebruiker\Desktop\IronHack\Labs\week 7\lab-natural-language-processing\data\kg_test.csv", encoding='latin-1')


In [6]:
data_test = data.head(200)


In [7]:
X_train = data_test['text']
Y_train = data_test['label']

x_test = data_test['text']

# confirm shape
print(f"Train size: {X_train.shape}, Test size: {x_test.shape}")

Train size: (200,), Test size: (200,)


## Data Preprocessing

In [8]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [105]:
!pip install beautifulsoup4


In [9]:
import re
from bs4 import BeautifulSoup
import pandas as pd

In [107]:
# Raw HTML = messy text with lots of tags
# Parsing HTML = converting that mess into a readable tree structure, - Reading and analyzing HTML code so a program can understand its structure and extract useful information
# Parser = tool that does this (e.g., html.parser, lxml, html5lib)

In [10]:
import re

def remove_html(text):
    # Remove HTML comments
    no_comments = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Remove HTML tags
    clean = re.sub(r'<[^>]+>', '', no_comments)
    # remove remaining HTML tags - removes any remaining HTML tags (e.g., <p>, <br>, <strong>) using regex
    #Regex breakdown: <: Start of an HTML tag, [^>]+: One or more characters that are NOT >, >: End of the tag, Leaves only plain text behind.
    
    # Normalize whitespace
    clean = re.sub(r'\s+', ' ', clean).strip()   
    # replace multiple spaces with a single space, \s+ matches any number of whitespace characters (spaces, newlines, tabs), replaces them with a single space,
    #.strip() removes leading/trailing spaces, makes the result tidy and consistent
    
    return clean


In [11]:
# Apply cleaning function
data['cleaned_text'] = data['text'].apply(remove_html)
data_test['cleaned_text'] = data_test['text'].apply(remove_html)


C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_8268\1638377541.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['cleaned_text'] = data_test['text'].apply(remove_html)


In [12]:
# Optional: preview
print(data[['text', 'cleaned_text', 'label']].head())       # OK for training data
print(data_test[['text', 'cleaned_text']].head())           # NO 'label' in test data


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                        cleaned_text  label  
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1  
1                                           Will do.      0  
2  Nora--Cheryl has emailed dozens of memos about...      0  
3  Dear Sir=2FMadam=2C I know that this proposal ...      1  
4                                                fyi      0  
                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4          

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [13]:
def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # keep only letters and whitespace

    # Remove all single characters (e.g., "a", "b")
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # Remove single characters from the start (e.g., "a This" → "This")
    text = re.sub(r'^[a-zA-Z]\s+', '', text)

    # Remove prefix "b" (in case of byte strings)
    text = re.sub(r'^b\s+', '', text)

    # Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)

    # Convert to lowercase
    return text.lower().strip()

# Apply preprocessing function to cleaned text
data['preprocessed_text'] = data['cleaned_text'].apply(preprocess_text)
data_test['preprocessed_text'] = data_test['cleaned_text'].apply(preprocess_text)

data[['text', 'cleaned_text', 'preprocessed_text']].head()


C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_8268\3924885812.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['preprocessed_text'] = data_test['cleaned_text'].apply(preprocess_text)


,text,cleaned_text,preprocessed_text
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...","DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",dear sir strictly private business proposal am...
1,Will do.,Will do.,will do
2,Nora--Cheryl has emailed dozens of memos about...,Nora--Cheryl has emailed dozens of memos about...,noracheryl has emailed dozens of memos about h...
3,Dear Sir=2FMadam=2C I know that this proposal ...,Dear Sir=2FMadam=2C I know that this proposal ...,dear sirfmadamc know that this proposal might ...
4,fyi,fyi,fyi


## Now let's work on removing stopwords
Remove the stopwords.

In [14]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

# Remove stop words
def remove_stopwords(text):
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gebruiker\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [15]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_words(text):
    words = text.split()
    lemmatized = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized)

In [16]:
data['lemmatized_text'] = data['preprocessed_text'].apply(lemmatize_words)
data_test['lemmatized_text'] = data_test['preprocessed_text'].apply(lemmatize_words)


C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_8268\1164293317.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['lemmatized_text'] = data_test['preprocessed_text'].apply(lemmatize_words)


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

# Convert processed_docs to strings
texts = [' '.join(doc) for doc in 'lemmatized_text']

texts = data['lemmatized_text'].tolist()
labels = data['label'].tolist()
# Create the Bag of Words model
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

# Convert to DataFrame so we can see the word counts
word_counts = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
word_counts['label'] = labels  # add the labels back

In [18]:
#Find top 10 words in ham
ham = word_counts[word_counts['label'] == 'ham']
top_ham = ham.drop(columns='label').sum().sort_values(ascending=False).head(10)
print("Top 10 HAM words:\n", top_ham)

Top 10 HAM words:
 zurich            0
aac               0
aaclocated        0
aae               0
aag               0
aaronovitchon     0
abacha            0
zimbabwei         0
zimbabwec         0
zimbabwebefore    0
dtype: int64


In [19]:
# Get top 10 SPAM words
top_spam = word_counts[word_counts['label'] == 'spam'].drop(columns='label').sum().sort_values(ascending=False).head(10)
print("\nTop 10 SPAM words:\n", top_spam)


Top 10 SPAM words:
 zurich            0
aac               0
aaclocated        0
aae               0
aag               0
aaronovitchon     0
abacha            0
zimbabwei         0
zimbabwec         0
zimbabwebefore    0
dtype: int64


## Extra features

In [118]:
#data = pd.read_csv(r"C:\Users\Gebruiker\Desktop\IronHack\Labs\week 7\lab-natural-language-processing\data\kg_train.csv", encoding='latin-1')

In [20]:
data_train = data.copy()

In [21]:
data_val = data_test.copy()
#adding features to the cleaned data test

In [22]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,cleaned_text,preprocessed_text,lemmatized_text,money_mark,suspicious_words,text_len
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",dear sir strictly private business proposal am...,dear sir strictly private business proposal am...,1,1,2195
1,Will do.,0,Will do.,will do,will do,1,0,7
2,Nora--Cheryl has emailed dozens of memos about...,0,Nora--Cheryl has emailed dozens of memos about...,noracheryl has emailed dozens of memos about h...,noracheryl ha emailed dozen of memo about hait...,1,0,192
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,Dear Sir=2FMadam=2C I know that this proposal ...,dear sirfmadamc know that this proposal might ...,dear sirfmadamc know that this proposal might ...,1,1,2017
4,fyi,0,fyi,fyi,fyi,1,0,3


## How would work the Bag of Words with Count Vectorizer concept?

In [23]:
vectorizer = CountVectorizer()

data_train = vectorizer.fit_transform(data_train['preprocessed_text'])
data_val = vectorizer.transform(data_val['preprocessed_text'])


In [24]:
print(vectorizer.get_feature_names_out())

['aac' 'aaclocated' 'aae' ... 'zumadirector' 'zumae' 'zurich']


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Vectorize all dataset
X_tfidf = tfidf_vectorizer.fit_transform(data['preprocessed_text'])  # Use full dataset or split 

# Print shape of the vectorized dataset
print("TF-IDF vectorized dataset shape:", X_tfidf.shape)


TF-IDF vectorized dataset shape: (1000, 20356)


## And the Train a Classifier?

In [40]:
from sklearn.model_selection import train_test_split
X_text = data['preprocessed_text']
y = data['label']

# Vectorize text
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X_text)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit model
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Predict on validation set
y_pred = clf.predict(X_val)

print("Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

Accuracy: 0.94
              precision    recall  f1-score   support

           0       0.92      0.98      0.95       125
           1       0.97      0.87      0.92        75

    accuracy                           0.94       200
   macro avg       0.95      0.93      0.93       200
weighted avg       0.94      0.94      0.94       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code